In [ ]:
# core modules
import tensorflow as tf
import numpy as np

# data module
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# auxiliary modules
import math
import os
import matplotlib.pyplot as plt
from PIL import Image

"""
******************* Data Prep *****************
original data set size: 25000
12500 cats, 12500 dogs   
microsoft asirre datasets cats and dogs (from kaggle competition)
***********************************************
"""

base_dir = 'Datasets'
base_dir = os.path.join(base_dir,'cats_and_dogs_25000')
training_dir =os.path.join(base_dir,'train')
validation_dir =os.path.join(base_dir,'validation')

train_dir_cats = os.path.join(training_dir,'cats')
train_dir_dogs = os.path.join(training_dir,'dogs')
validation_dir_cats = os.path.join(validation_dir,'cats')
validation_dir_dogs = os.path.join(validation_dir,'dogs')

print(base_dir)
print(training_dir)
print(validation_dir)
print(train_dir_cats)
print(validation_dir_cats)

# check basic statistics and number of files
num_train_cats=len(os.listdir(train_dir_cats))
print("Number of training files for cats : {}".format(num_train_cats))
num_train_dogs=len(os.listdir(train_dir_dogs))
print("Number of training files for dogs : {}".format(num_train_dogs))
num_val_cats = len(os.listdir(validation_dir_cats))
num_val_dogs = len(os.listdir(validation_dir_dogs))
total_train=num_train_cats + num_train_dogs
total_val = num_val_cats+num_val_dogs

"""
**************************  Image Preparation ****************
Images must be formatted into pre-processed floating point tensors before being fed into the network

Steps:
Read files from disk
Decode the contents of these images and convert them into a proper grid format as per their RGB content
Convert them to floating point tensors
Rescale the tensors from values between 0 and 255 to values between 0 and 1 
(since NN work better with small input values) 
These can be done with the : tensorflow.keras.preprocessing.image.ImageDataGenerator
"""

BATCH_SIZE = 100
IMAGE_SHAPE =150
training_image_generator = ImageDataGenerator(rescale =1/255)
validation_image_generator = ImageDataGenerator(rescale =1/255)

train_gen = training_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                        directory=training_dir,shuffle=True,target_size=(IMAGE_SHAPE,IMAGE_SHAPE),class_mode='binary')
validation_gen = validation_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                        directory=validation_dir,shuffle=True,target_size=(IMAGE_SHAPE,IMAGE_SHAPE),class_mode='binary')



"""
NETWORK NETWORK DESIGN

"""

layer0 = tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3))
layer1 = tf.keras.layers.MaxPooling2D(2,2)
layer2 = tf.keras.layers.Conv2D(64,(3,3),activation='relu')
layer3 = tf.keras.layers.MaxPooling2D(2,2)
layer4 = tf.keras.layers.Conv2D(128,(3,3),activation='relu')
layer5 = tf.keras.layers.MaxPooling2D(2,2)
layer6 = tf.keras.layers.Flatten()
layer7 = tf.keras.layers.Dense(512,activation='relu')
layer8 = tf.keras.layers.Dense(2,activation='softmax')

model =tf.keras.Sequential([layer0,layer1,layer2,layer3,layer4,layer5,layer6,layer7,layer8])

model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

model.summary()

history = model.fit(train_gen,steps_per_epoch=math.ceil(total_train/BATCH_SIZE),epochs=100,validation_data=validation_gen,validation_steps=math.ceil(total_val/BATCH_SIZE))
test_loss,test_accuracy = model.evaluate(validation_gen,steps=math.ceil(total_train/BATCH_SIZE))

print("Model accuracy: {}".format(test_accuracy))
